In [ ]:
import boto3

r53_client = boto3.client('route53', region_name='us-west-1')

In [ ]:
# Get all the hosted zones
response = r53_client.list_hosted_zones()
hosted_zones = response['HostedZones']
print hosted_zones

In [ ]:
# Extract the hosted zone for a specific domain
dev_center = [hz for hz in hosted_zones if hz['Name'] == 'xtdevcentre.net.'][0]
print dev_center

In [ ]:
# List the record sets for a hosted zone
response = r53_client.list_resource_record_sets(
    HostedZoneId=dev_center['Id']
)

print response

In [ ]:
# Look at the records sets in the hosted zone - you should see an NS record, and SOA record, and an Alias per
# specific subdomain assuming you've been creating aliases for subdomains, for example at the load balancer level.

for rr in response['ResourceRecordSets']:
    print rr



In [ ]:
elbv2_client = boto3.client('elbv2', region_name='us-west-1')
response = elbv2_client.describe_load_balancers()
print response

In [ ]:
arn = response['LoadBalancers'][0]['LoadBalancerArn']
print arn

In [ ]:
response = elbv2_client.describe_load_balancer_attributes(
    LoadBalancerArn=arn
)

print response

## Route 53 Health Check - API Gateway Endpoint



In [ ]:
#https://y3eeprh0j9.execute-api.us-east-1.amazonaws.com/hc1/todos/health
import boto3
import uuid

client = boto3.client('route53')


response = client.create_health_check(
    
    CallerReference=str(uuid.uuid4()),
    HealthCheckConfig={
        'Type':'HTTPS',
        'ResourcePath':'/hc1/todos/health',
        'FullyQualifiedDomainName':'y3eeprh0j9.execute-api.us-east-1.amazonaws.com'
    }
)

print response
        

In [ ]:
hc_id = response['HealthCheck']['Id']

In [ ]:
tag_resp = client.change_tags_for_resource(
    ResourceType='healthcheck',
    ResourceId=hc_id,
    AddTags=[
        {
            'Key':'Name',
            'Value':'SampleHC'
        },
    ]
)

print tag_resp

In [ ]:
response = client.delete_health_check(
    HealthCheckId=hc_id
)

In [ ]:
print response